In [ ]:
import numpy as np

In [ ]:
from ipynb.fs.defs.get_data import get_confirmed_planets
exoplanet_archive_table = get_confirmed_planets(select="*")

In [ ]:
exoplanet_archive_table.colnames

In [ ]:
filter = (exoplanet_archive_table['pl_bmasse'] > 0.0) \
        & (exoplanet_archive_table['pl_rade'] > 0.0)

In [ ]:
print("There are {0} confirmed planets with measured mass + radius.".format(
    len(exoplanet_archive_table[filter])))

In [ ]:
from bokeh.plotting import ColumnDataSource, figure, show
from bokeh.io import output_notebook
output_notebook()

In [ ]:
table = exoplanet_archive_table[filter]
source = ColumnDataSource(table.to_pandas())

In [ ]:
fig = figure(tools="pan,wheel_zoom,box_zoom,reset", active_scroll="wheel_zoom",
            x_axis_type="linear", x_range=[0.0, 20.0], y_range=[0.0,5.0])

In [ ]:
pl_render = fig.circle('pl_bmasse','pl_rade', source=source, size=10,color='firebrick')
fig.background_fill_color = "beige"
fig.background_fill_alpha = 0.5
fig.plot_height=900
fig.plot_width=900
show(fig)

In [ ]:
from bokeh.models import HoverTool
hover = HoverTool(renderers=[pl_render],
                    tooltips=[
        ("name", "@pl_name"),
        ("mass", "@pl_bmasse{1.11} Earth masses"),
        ("radius", "@pl_rade{1.11} Earth radius"),
        ("discovered by", "@pl_discmethod")
        ]
    )
fig.add_tools(hover)

In [ ]:
show(fig)

In [ ]:
err_scale = (table['pl_masseerr1'] + table['pl_masseerr2'])**2/table['pl_bmasse']**2 \
                    + (table['pl_radeerr1'] + table['pl_radeerr2'])**2/table['pl_rade']**2 
err_weight = np.exp(-err_scale**0.15) # trial & error

In [ ]:
source.add(err_weight, name='err_weight')
pl_render.data_source = source # update source
pl_render.glyph.fill_alpha = 'err_weight'

In [ ]:
r_err_xs, r_err_ys = [], []
m_err_xs, m_err_ys = [], []
for pl in table:
    m = pl['pl_bmasse']
    r = pl['pl_rade']
    r_err_xs.append((m, m))
    r_err_ys.append((r + pl['pl_radeerr1'], r + pl['pl_radeerr2']))
    m_err_xs.append((m + pl['pl_masseerr1'], m + pl['pl_masseerr2']))
    m_err_ys.append((r, r))
fig.multi_line(r_err_xs, r_err_ys, line_alpha=err_weight)
fig.multi_line(m_err_xs, m_err_ys, line_alpha=err_weight)

In [ ]:
fig.xaxis.axis_label = 'Mass (Earth Masses)'
fig.yaxis.axis_label = 'Radius (Earth Radii)'
fig.xaxis.axis_label_text_font_size = '14pt'
fig.xaxis.major_label_text_font_size = '12pt'
fig.yaxis.axis_label_text_font_size = '14pt'   
fig.yaxis.major_label_text_font_size = '12pt' 
fig.toolbar_location = "above"

In [ ]:
show(fig)

In [ ]:
from bokeh.models import TapTool, OpenURL
fig.add_tools(TapTool())
url = "@pl_pelink"
taptool = fig.select(type=TapTool)
taptool.callback = OpenURL(url=url)

In [ ]:
show(fig)

In [ ]:
from bokeh.plotting import output_file, save
output_file('exoplanet_massradius.html')
save(fig)

In [ ]:
import altair as alt
table.add_column(err_weight, name='err_weight')
data = table.to_pandas()

In [ ]:
alt.renderers.enable('notebook')

In [ ]:
points = alt.Chart(data).mark_point().encode(
    x='pl_bmasse:Q',
    y='pl_rade:Q',
    opacity='err_weight:Q'
).interactive()

In [ ]:
points

In [ ]:
points = alt.Chart(data).mark_point(clip=True, filled=True).encode(
    x=alt.X('pl_bmasse:Q', scale=alt.Scale(domain=(0, 20)),
           axis=alt.Axis(title='Mass (Earth Masses)')),
    y=alt.Y('pl_rade:Q', scale=alt.Scale(domain=(0, 5)),
           axis=alt.Axis(title='Radius (Earth Radii)')),
    opacity=alt.Opacity('err_weight:Q', legend=None)
).interactive()

In [ ]:
points

In [ ]:
data['mass_err1'] = data['pl_bmasse'] + data['pl_masseerr1']
data['mass_err2'] = data['pl_bmasse'] + data['pl_masseerr2']
data['rad_err1'] = data['pl_rade'] + data['pl_radeerr1']
data['rad_err2'] = data['pl_rade'] + data['pl_radeerr2']

In [ ]:
mass_error = alt.Chart(data).mark_rule().encode(
    x='mass_err1:Q',
    x2='mass_err2:Q',
    y='pl_rade:Q',
    color=alt.ColorValue('grey'),
    opacity=alt.Opacity('err_weight:Q', legend=None)
)

rad_error = alt.Chart(data).mark_rule().encode(
    x='pl_bmasse:Q',
    y='rad_err1:Q',
    y2='rad_err2:Q',
    color=alt.ColorValue('grey'),
    opacity=alt.Opacity('err_weight:Q', legend=None)
)

mass_error + rad_error + points

In [ ]:
data['st_abs_mag'] = data['gaia_gmag'] - 5.*(np.log10(data['gaia_dist']) - 1.)

In [ ]:
brush = alt.selection(type='interval', resolve='global')

planets = alt.Chart(data).mark_point(clip=True, filled=True, size=28).encode(
    x=alt.X('pl_bmasse:Q', scale=alt.Scale(domain=(0, 20)),
           axis=alt.Axis(title='Mass (Earth Masses)')),
    y=alt.Y('pl_rade:Q', scale=alt.Scale(domain=(0, 5)),
           axis=alt.Axis(title='Radius (Earth Radii)')),
    opacity=alt.Opacity('err_weight:Q', legend=None),
    color=alt.condition(brush, alt.ColorValue('blue'), alt.ColorValue('gray'))
).properties(
    selection=brush,
    width=300,
    height=300
)


stars = alt.Chart(data).mark_point(clip=True, filled=True, size=28).encode(
    x=alt.X('st_teff:Q', scale=alt.Scale(domain=(12e3, 3e3)),
           axis=alt.Axis(title='Effective Temperature (K)')),
    y=alt.Y('st_abs_mag:Q', scale=alt.Scale(domain=(15, -5)),
           axis=alt.Axis(title='Abs. G Magnitude')),
    opacity=alt.condition(brush, alt.OpacityValue(1.), alt.OpacityValue(0.4)),
    color=alt.condition(brush, alt.ColorValue('blue'), alt.ColorValue('gray'))
).properties(
    selection=brush,
    width=300,
    height=300
)

In [ ]:
planets | stars

In [ ]:
brush = alt.selection(type='interval', resolve='global', 
                          on="[mousedown[event.shiftKey], window:mouseup] > \
                          window:mousemove!", zoom='False',
                          translate="[mousedown[event.shiftKey], window:mouseup] > \
                          window:mousemove!")

pan = alt.selection(type='interval', bind='scales',
                        on="[mousedown[!event.shiftKey], window:mouseup] > \
                        window:mousemove!",
                        translate="[mousedown[!event.shiftKey], window:mouseup] > \
                        window:mousemove!")

planets = alt.Chart(data).mark_point(clip=True, filled=True, size=28).encode(
    x=alt.X('pl_bmasse:Q', scale=alt.Scale(domain=(0, 20)),
           axis=alt.Axis(title='Mass (Earth Masses)')),
    y=alt.Y('pl_rade:Q', scale=alt.Scale(domain=(0, 5)),
           axis=alt.Axis(title='Radius (Earth Radii)')),
    opacity=alt.Opacity('err_weight:Q', legend=None),
    color=alt.condition(brush, alt.ColorValue('blue'), alt.ColorValue('gray'))
).properties(
    selection=brush+pan,
    width=300,
    height=300
)


stars = alt.Chart(data).mark_point(clip=True, filled=True, size=28).encode(
    x=alt.X('st_teff:Q', scale=alt.Scale(domain=(12e3, 3e3)),
           axis=alt.Axis(title='Effective Temperature (K)')),
    y=alt.Y('st_abs_mag:Q', scale=alt.Scale(domain=(15, -5)),
           axis=alt.Axis(title='Abs. G Magnitude')),
    opacity=alt.condition(brush, alt.OpacityValue(1.), alt.OpacityValue(0.4)),
    color=alt.condition(brush, alt.ColorValue('blue'), alt.ColorValue('gray'))
).properties(
    selection=brush+pan,
    width=300,
    height=300
)

In [ ]:
planets | stars